# Union Find

As applied to the dynamic connectivity problem. 

* quick find
* quick union
* weighted QU
* weighted QU with path compression 

## Dynamic connectivity

Given a set of __nodes__ 0 to N-1, and various paths __connecting__ those nodes, are nodes p and q connected?

Connections are:
* reflexive (p C p)
* symmetric (p C q iff q C p)
* transitive (p C q and q C r => p C r)

a set of connected nodes form a __connected component__

`{0} {1 4 5} {2 3 6 7}`

### API

* UF(N): initialise a UF datastructure with n nodes
* union(p q): add connection between p and q
* connected(p q): are p and q connected?
* find(q): component identifier
* count(): number of components

## Quick Find

An eager approach.

Data structure: an array of integers `id`. p and q are connected iff they have the same id.

* connected: check if p and q have the same id
* union: change all entries whose id equals `id[p]` to `id[q]`

In [21]:
class QuickFind:
    def __init__(self, n):
        self.ids = [i for i in range(n)]
        
    def connected(self, p, q):
        return self.ids[p] == self.ids[q]
    
    def union(self, p, q):
        old = self.ids[p]
        new = self.ids[q]
        
        for i in range(len(self.ids)):
            if self.ids[i] == old:
                self.ids[i] = new
        
qf = QuickFind(10)
qf.union(0, 1)
qf.union(1, 2)
print(qf.connected(0, 2))
print(qf.connected(0, 3))

True
False


### Quick find performance

* construction: set the array entry to itself: N array accesses
* connected: 2 array accesses
* union: change all all entries with id[p] to id[q] - max 2N + 2 accesses (why?)

This is TOO SLOW. N^2 array accesses to process N union commands on N objects

## Quick Union

Data structure is again an array, but we use it to represent a tree, where the array entry `id[i]` is the __parent__ node of node i.

__root__ of i is `id[id[id[...id[i]...]]]` (keep going until id[i] = i, i.e. a node is its own parent)

```
     0 1 2 3 4 5 6 7 8 9
id[] 0 1 9 4 9 6 6 7 8 9


0  1  9    6  7  8 
      /\   |
     2  4  5
        |
        3
```

connected: check if root of p and root of q are the same

union: set the id of p's root to the id of q's root

In [50]:
class QuickUnion:
    def __init__(self, n):
        self.ids = [i for i in range(n)]
        
    def root(self, p):
        i = self.ids[p]
        
        while i != self.ids[i]:
            i = self.ids[i]
            
        return i
        
    def connected(self, p, q):
        return self.root(p) == self.root(q)
    
    def union(self, p, q):
        old_root = self.root(p)
        new_root = self.root(q)
        
        self.ids[old_root] = new_root
        
qu = QuickUnion(10)
print(qu.ids)
qu.union(2, 9)
qu.union(3, 4)
qu.union(4, 9)
qu.union(5, 6)
qu.union(9, 6)
print(qu.ids)
print(qu.connected(3, 2))
print(qu.connected(6, 7))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 9, 4, 9, 6, 6, 7, 8, 6]
True
False


### Quick union performance

* construction: same as before: N array accesses
* root: x accesses, where x is the depth of the path (i.e. N in the worst case)
* connected: x + y accesses, where x and y are the depths of the two paths
* union: x + y accesses, where x and y are the depths of the two paths

| algorithm   | init | union | find |
|-------------|------|-------|------|
| Quickfind   |  N   |   N   |  1   |
| QuickUnion  |  N   |   N   |  N   |


This is still TOO SLOW. Union is faster than QF, but find is slower.

The problem is that when trees can get tall, and that makes both union and find more expensive. We can mitigate that by weighting the tree

## Weighted quick union

We want to make each tree as 'short' as possible. When we union two trees together, the one that is attached, the 'depth' of every node is increased by one. So we can minimise the overall increase in tree depth when we union by attaching the tree the the fewer nodes to the one with most.

We keep track of the size of each tree with another array.

In [57]:
class WeightedQuickUnion:
    def __init__(self, n):
        self.parents = [i for i in range(n)]
        self.size = [1 for i in range(n)]
        
    def root(self, p):
        i = self.parents[p]
        
        while i != self.parents[i]:
            i = self.parents[i]
            
        return i
        
    def connected(self, p, q):
        return self.root(p) == self.root(q)
    
    def union(self, p, q):
        p_root = self.root(p)
        q_root = self.root(q)
        
        if p_root == q_root:
            pass
        
        if self.size[p_root] < self.size[q_root]:
            self.parents[p_root] = q_root
            self.size[q_root] += self.size[p_root]
        else:
            self.parents[q_root] = p_root
            self.size[p_root] += self.size[q_root]
        
qu = WeightedQuickUnion(10)
print(qu.parents)
qu.union(2, 9)
qu.union(3, 4)
qu.union(4, 9)
qu.union(5, 6)
qu.union(9, 6)
print(qu.connected(3, 2))
print(qu.connected(6, 7))
print(qu.parents)
print(qu.size)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
True
False
[0, 1, 3, 3, 3, 3, 5, 7, 8, 2]
[1, 1, 2, 6, 1, 2, 1, 1, 1, 1]


What we've done is limit the max depth of any node to `lg N` (where lg is base 2 logarithm)


| algorithm   | init | union | find |
|-------------|------|-------|------|
| Quickfind   |  N   |   N   |  1   |
| QuickUnion  |  N   |   N   |  N   |
| WeightedQuickUnion  |  N   |   lg N   |  lg N   |

## WQU with path compression

We want to minimize tree depth, so the final step is reduce the tree size as you traverse it - path compression.

When you examine a node, and it's parent is not the tree root, set the parent of the node to the root. Do the same thing for every node you touch.

You could add a second loop which, for each node you look at, finds the root, and sets the parent to that. This is the _two pass implementation_

The _one pass implementation_ is to just move the parent of a node to it's grandparent

This is actually only one extra line of code:

`self.parents[i] = self.parents[self.parents[i]]`

In [59]:
class WeightedQuickUnion:
    def __init__(self, n):
        self.parents = [i for i in range(n)]
        self.size = [1 for i in range(n)]
        
    def root(self, p):
        i = self.parents[p]
        
        while i != self.parents[i]:
            self.parents[i] = self.parents[self.parents[i]]
            i = self.parents[i]
            
        return i
        
    def connected(self, p, q):
        return self.root(p) == self.root(q)
    
    def union(self, p, q):
        p_root = self.root(p)
        q_root = self.root(q)
        
        if p_root == q_root:
            pass
        
        if self.size[p_root] < self.size[q_root]:
            self.parents[p_root] = q_root
            self.size[q_root] += self.size[p_root]
        else:
            self.parents[q_root] = p_root
            self.size[p_root] += self.size[q_root]
        
qu = WeightedQuickUnion(10)
print(qu.parents)
qu.union(2, 9)
qu.union(3, 4)
qu.union(4, 9)
qu.union(5, 6)
qu.union(9, 6)
print(qu.connected(3, 2))
print(qu.connected(6, 7))
print(qu.parents)
print(qu.size)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
True
False
[0, 1, 3, 3, 3, 3, 5, 7, 8, 2]
[1, 1, 2, 6, 1, 2, 1, 1, 1, 1]


A problem with _M_ union-find operations on _N_ objects will take:

* quick-find: M N
* quick-union: M N
* wqu: N + M log N
* qu + pc: N + M log N
* wqu + pc: N + M lg* N

lg* N is, in practice, linear time.

The consequences of this are that an problem with 10^9 unions-find operations on 10^9 objects is reduced from 30 years to 6 seconds. A good illustration of where better algorithms will help, where more computing power won't